# A First Neural Network Using Keras

This second notebook goes further and explores neural networks with Keras. This time I'll use image data only.

**Objectives**

1. Import image data
2. Prepare data for feeding a neural network
3. Build, train and evaluate a Keras neural network
4. Submit results for ranking

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pylab as plt
import os
import PIL

## Enable GPU

In [ ]:
print('TensorFlow version: {}'.format(tf.__version__))
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('GPU device not found - On for CPU time!')
else:
    print('Found GPU at {}'.format(device_name))

## ETL: load data and prepare it for feeding a Keras model

First of all, we must properly handle the material our model will have to deal with for training. Training images are located in `'../input/petfinder-pawpularity-score/train/'`. Let's check some attributes of the images, such as their count, and their size.

### Image attributes analysis

In [ ]:
#path = '../input/petfinder-pawpularity-score/train/'
#training_img = os.listdir(path) # list all training images names
#print('There are {} images in the training directory'.format(len(training_img)))

#img_sz = {'width': list(),
#          'height': list()} # store image attributes for further analysis

#for im in training_img:
#    img = PIL.Image.open(path+im)
#    w, h = img.size
#    img_sz['width'].append(w)
#    img_sz['height'].append(h)

#IMG_WIDTH = tf.math.reduce_mean(img_sz['width'])
#IMG_HEIGHT = tf.math.reduce_mean(img_sz['height'])
#IMG_CHANNELS = 3

#print('Average training image width: {} px'.format(IMG_WIDTH))
#print('Average training image height: {} px'.format(IMG_HEIGHT))

### Data handlers

In [ ]:
# Let's define some helpers

IMG_HEIGHT = 150 # Let's try to arbitrarily set 150px x 150px images
IMG_WIDTH = 150
IMG_CHANNELS = 3

def read_and_decode(filename, reshape_dims):
    # Read an image file to a tensor as a sequence of bytes
    img = tf.io.read_file(filename)
    # Convert the tensor to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=IMG_CHANNELS)
    # Convert 3D uint8 tensor 
    img = tf.image.convert_image_dtype(img, tf.float32)
    # Resize the image to the desired size
    return tf.image.resize(img, reshape_dims)

def show_image(filename):
    img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
    plt.imshow(img.numpy());
    plt.axis('off');
    
def decode_csv(csv_row):
    record_defaults = ['Id', 'Pawpularity']
    filename, pawpularity = tf.io.decode_csv(csv_row, record_defaults)
    pawpularity = tf.convert_to_tensor(np.float(pawpularity), dtype=tf.float32)
    img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
    return img, pawpularity

### Training and test sets

Here we'll ensure that training and test sets are built from the same distribution by using stratified sampling instead of random sampling.

In [ ]:
data_path = '../input/petfinder-pawpularity-score/'
data = pd.read_csv(data_path+'train.csv')

# Use stratified sampling
sssplit = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
for train_index, test_index in sssplit.split(data, data['Pawpularity']):
    training_set = data.iloc[train_index]
    eval_set = data.iloc[test_index]
    
# Visually check distribution of pawpularity score in training and test sets
training_set['Pawpularity'].hist(label='Training set')
eval_set['Pawpularity'].hist(label='Eval set')
plt.title('Pawpularity score distribution in training and test set')
plt.xlabel('Pawpularity score')
plt.ylabel('Count')
plt.legend(loc='upper right')
plt.show()

# Export training and test sets as .csv files
training_set['Id'] = training_set['Id'].apply(lambda x: '../input/petfinder-pawpularity-score/train/'+x+'.jpg')
training_set[['Id', 'Pawpularity']].to_csv('/kaggle/working/training_set.csv', header=False, index=False)
eval_set['Id'] = eval_set['Id'].apply(lambda x: '../input/petfinder-pawpularity-score/train/'+x+'.jpg')
eval_set[['Id', 'Pawpularity']].to_csv('/kaggle/working/eval_set.csv', header=False, index=False)

## Neural network using Keras

First neural network is built with a single hidden layer and no further optimisation or hyperparameter tuning

In [ ]:
BATCH_SIZE = 32
IMG_HEIGHT = 150 # Let's try to arbitrarily set 150px x 150px images
IMG_WIDTH = 150
IMG_CHANNELS = 3

train_dataset = tf.data.TextLineDataset(
    '/kaggle/working/training_set.csv'
).map(decode_csv).batch(BATCH_SIZE)

eval_dataset = tf.data.TextLineDataset(
    '/kaggle/working/eval_set.csv'
).map(decode_csv).batch(BATCH_SIZE)

# Our neural network is built as a Sequential model with a single hidden layer
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
    tf.keras.layers.Dense(units=128, activation='relu'), # The hidden layer adds nonlinearity with the ReLU activation function
    tf.keras.layers.Dense(units=1, activation=None)
])

# Let's compile it with Adam, a well-suited optimiser for CV problems
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

# And let's now train our model with the training and evaluation data
history = model.fit(train_dataset, validation_data=eval_dataset, epochs=10)

In [ ]:
# Let's plot our neural network to see how data is passed through
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=False)

## Compute predictions and build submission process

In [ ]:
sample_submission = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
sample_submission['Id'] = sample_submission['Id'].apply(lambda x: '../input/petfinder-pawpularity-score/test/'+x+'.jpg')
sample_submission.to_csv('/kaggle/working/sample_submission.csv', index=False, header=False)
sample_submission = tf.data.TextLineDataset(
    './sample_submission.csv'
).map(decode_csv).batch(BATCH_SIZE)

# Make predictions with our model
sample_prediction = model.predict(sample_submission)

In [ ]:
# Format predictions to output for submission
submission_output = pd.concat(
    [pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv').drop('Pawpularity', axis=1),
    pd.DataFrame(sample_prediction)],
    axis=1
)
submission_output.columns = [['Id', 'Pawpularity']]

# Output submission file to csv
submission_output.to_csv('submission.csv', index=False)